In [ ]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
## 1
dateparse = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f')
df = pd.read_csv('2019-5-8 2019-5-9_pivot.csv', sep = ';', parse_dates=['timestamp'], date_parser=dateparse, index_col='timestamp')

cols = list(df)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('SM1281_Status_module1_Channel_1_aRMS')))
df = df.loc[:, cols]


In [ ]:
## 2
print('number of rows is %s' % len(df.index))
print('number of columns is %s' % len(cols))
print('start datetime is %s' % df.index[0])
print('end datetime is %s' % df.index[len(df.index) - 1])
df.info()
df.shape
print(df.isnull().sum()) # print per coll the nb of values NaN or None


In [ ]:
## 2.c
f = ~df.isin([np.nan, 'NA', 0]).all()
# print(f)
df = df.loc[:,f]
cols = list(df)
print('number of columns is %s' % len(cols))


In [ ]:
## 2.d
for i in df.columns:
    if df[i].dtype == object:
        continue
    plt.figure(figsize=(16,8))
    plt.plot(df[i])
    plt.title(i)

In [ ]:
## 2.e
def get_highly_corr_columns (df):
    corr_mat = df.corr(method='pearson')
    upper_corr_mat = corr_mat.where(
        np.triu(np.ones(corr_mat.shape), k=1).astype(bool))
  
    # Convert to 1-D series and drop Null values
    unique_corr_pairs = upper_corr_mat.unstack().dropna()

    # Sort correlation pairs
    sorted_mat = unique_corr_pairs.sort_values()
    return sorted_mat.tail(40)

get_highly_corr_columns(df)


In [ ]:
## 3
def fill_missing_values (df):
    for i in df.columns:
        if df[i].dtype == object:
            continue
        df[i].interpolate(inplace=True)
        df[i].fillna(inplace=True, method='backfill')
        df[i].fillna(df[i].mean(), inplace=True)
fill_missing_values(df)
df

In [ ]:
## 4
"""
a: Define train data and test data. Further clean the data, only keep data of when the machine is running.
b: PCA and perform anomaly detection on that data. PCA because we can reduce the number of dimensions that way.
c: The performance will be tested with labeled data of failures and non failures. See how many postive and false-positive there are.
Also check the time to compute and the failure prediction time.
d: If would focus on vibrations, speeds, torque and temperatures. For those I would try to make some baseline for when the machine runs normally.
With those baselines for normal activity I would try to detect anomalies when for a sustained amount of time the values are over the thershold.
"""